# Statistical Machine Learning -- Notebook 8, version for students
**Author: Dorota Celińska-Kopczyńska, Michał Ciach**  


## Description


In today's class, we will work with the model's diagnostics. We will be checking whether the linear regression assumptions are satisfied in our sample datasets. We will also discuss the individual and joint significance of the parameters in our model and look for any influential observations.

The importance of the diagnostics depends on the focus of our model. Generally, while working with linear regression, we may be primarily interested in either a) interpretability or b) prediction. If our main focus is interpretability, we have to pay special attention to the properties of the OLS estimator such as unbiasedness or efficiency. In such a situation, diagnostics is a must, i.e., if some assumptions are not satisfied in our model, the interpretability will be questionable. On the other hand, even a model that does not satisfy all assumptions may be beneficial for prediction.  

The assumptions we will check can be summarized as the LINE rule:    

- **L**inear trend,
- **I**ndependent residuals (lack of autocorrelation),
- **N**ormally distributed residuals,
- **E**qual variance of residuals for all values of independent variables (homoscedasticity).

We will check them visually by creating and analyzing the following diagnostic plots:   

- The residual value vs the fitted value,
- The root square of the absolute value of standardized residuals vs the fitted value,
- The line plot of the residuals,
- Graphical analysis of the distribution of the residuals (histogram, boxplots, qq-plot).

The first plot is used to check if the relationship between the response (the dependent variable) and the predictors (the independent variables) is linear and to very roughly check if the residuals are uncorrelated. We expect values distributed symmetrically across the line $y=0$. However, this plot may be misleading if a non-spherical random disturbance occurs. That is why we encourage performing the Ramsey RESET test.  

The second plot checks the variance's homoscedasticity (equality for all values of the independent variables). We expect values to be distributed symmetrically across a straight horizontal line.

The third plot allows you to determine if your model has problems with autocorrelation (or heteroscedasticity). We expect to see a plot resembling a sound wave of constant amplitude (homoscedasticity) and no trend (lack of autocorrelation).

The histogram is used to visualize the distribution of residuals. You can also use a qq-plot if you know how to create and interpret it.

While the above are the assumptions of the classical linear regression model, some problems with data might still occur and may significantly reduce the quality of the results obtained with OLS. That is why, in the later part of this notebook, we will also address the issues of multicollinearity and influential observations and how to find them. For the detection of the multicollinearity, we will use VIF. Finally, we will inspect either the influence plot or the leverage-resid2 plot,  implemented in [`statsmodels`](https://www.statsmodels.org/dev/examples/notebooks/generated/regression_plots.html). Both plots are used to detect outliers that highly influence the model parameters.

In [ ]:
!pip install gdown
!pip install --upgrade gdown
!gdown https://drive.google.com/uc?id=1PjeSdrN9E0fzs3J0zVT7P7a1PT1vMDZ6
!gdown https://drive.google.com/uc?id=1vne1N6D0yov8lrp9kCo1qZoT61CXYL6P
!gdown https://drive.google.com/uc?id=1k0iAQYLdppDQUBmKPZ0ckSNLNv9izwZU

## Data & library imports

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import statsmodels.api as sm
from scipy.stats import multivariate_normal as mvn
from scipy.linalg import svd

In [ ]:
income = pd.read_csv('6. BDL municipality incomes 2015-2020.csv', sep=';', dtype={'Code': 'str'})
population = pd.read_csv('6. BDL municipality population 2015-2020.csv', sep='\t', dtype={'Code': 'str'})
area = pd.read_csv('6. BDL municipality area km2 2015-2020.csv', sep='\t', dtype={'Code': 'str'})

In [ ]:
voivodeship_names = {
    '02': 'Dolnośląskie',
    '04': 'Kujawsko-pomorskie',
    '06': 'Lubelskie',
    '08': 'Lubuskie',
    '10': 'Łódzkie',
    '12': 'Małopolskie',
    '14': 'Mazowieckie',
    '16': 'Opolskie',
    '18': 'Podkarpackie',
    '20': 'Podlaskie',
    '22': 'Pomorskie',
    '24': 'Śląskie',
    '26': 'Świętokrzyskie',
    '28': 'Warmińsko-mazurskie',
    '30': 'Wielkopolskie',
    '32': 'Zachodniopomorskie'
}

In [ ]:
code_list = [s[:2] for s in income["Code"]]
name_list = [voivodeship_names[code] for code in code_list]
income['Voivodeship'] = name_list

## Diagnostics when assumptions are satisfied

**Exercise 1.** In this exercise, we will inspect the diagnostics of the model when the assumptions of the linear regression model are satisfied. We will focus on artificial data with a given multidimensional distribution and correlation matrix. Our variables are not highly correlated with each other. We consider a model in which y will be the dependent variable (response), and A, B, C, and the constant will be the independent ones (predictors).

The code below shows you how to generate data from a multivariate normal distribution. In [scipy.stats.multivariate_normal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multivariate_normal.html), you will find the function `rvs()`, which samples from a multivariate normal distribution. We need to specify the means of the variables and the covariance matrix (note that we provide variances on the diagonal of the matrix). For the replicability of the results, we also specify the seed -- this way every time you run this code, you will get exactly same result.

In [ ]:
# generate the artificial data -- X matrix
# construct the covariance matrix
cov = np.array([[1.0, 0.0, 0.0],[0.0,25.0,4.0],[0.0,4.0,25.0]])
cov

# set the seed for reproducibility
np.random.seed(17042023)

df = mvn.rvs(mean = [0.0,20.0,20.0], cov=cov, size = 1000)

X = pd.DataFrame(df, columns=['A', 'B', 'C'])
X

Our data are generated according to the linear model: $y = 0.4A + 0.5B + 0.5C + \varepsilon$. Note that this will be our ground truth -- the scenario we know is true, and we will compare against.

In [ ]:
# generate artificial data -- Y
Y = 0.4*X['A'] + 0.5*X['B'] + 0.5*X['C'] + np.random.normal(0,1,1000)
Y = pd.DataFrame(Y, columns=['Y'])
Y

**Exercise 1a**. Inspect the descriptive statistics of the data set.
Visualize the relationships between dependent and independent variables.

*Hint:* for a convenient preparation of scatterplots for all of the pairs of the variables in your dataset, you may use [`scatter_matrix`](https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter_matrix.html) from `plotly.express`.

In [ ]:
# put your code here


**Exercise 1b** Estimate the model and inspect its summary. Are the variables jointly significant according to the F-test? Are all individual variables significant according to the t-test? Compare the estimates' values with the parameters' true values (from the ground truth).

Hint: Not that the intercept is not included by default and we need to modify our dataset with [sm.add_constant()](https://www.statsmodels.org/dev/generated/statsmodels.tools.tools.add_constant.html).

In [ ]:
# put your code here


**Exercise 1c** Inspect the output of [get_prediction().summary_frame()](https://www.statsmodels.org/stable/generated/statsmodels.tsa.base.prediction.PredictionResults.summary_frame.html). Do you know what is in the `mean` and `mean_se` columns? Do you understand the difference `mean_ci` and `obs_ci` (*Mean Confidence Interval* and *Observation Prediction Interval*)?

Use the summary frame data to compute the residuals $\hat{\epsilon}_i = Y_i - X_i\hat{\beta}$. Calculate the standardized residuals $(\hat{\epsilon}_i - \text{mean}(\hat{\epsilon}))/\text{sd}(\hat{\epsilon})$, and square roots of absolute values of standardised residuals.

In [ ]:
# put your code here


**Linear functional form**

Based on the ground truth model that we used (we know how $y$ was generated), we know that the linear functional form assumption is satisfied.

In practice we would use the [Ramsey RESET test](https://en.wikipedia.org/wiki/Ramsey_RESET_test) (Regression Specification Error Test) to check that the model's functional form is correct, with:

$$H_0:X\beta+\varepsilon$$
$$H_1:f(X\beta)+\varepsilon$$

where f() is non-linear. Important: The functional form's linearity applies to the coefficients' powers; the x-variables can be transformed freely. There are several versions of this test. The version presented here estimates the original regression and obtains the fitted values. An auxiliary regression is then estimated in which successive powers of the fitted values from the primary regression become the independent variables. We test for joint insignificance of the estimated coefficients at powers greater than 1 of the fitted values in the auxiliary regression.

Interestingly, a similar idea to the RESET test can be used to test the functional form in other models. A generalized version of this test is called the link test.

In [ ]:
# Results of RESET test
print(sm.stats.diagnostic.linear_reset(results, power=3, test_type='fitted'))

The p-value in the RESET test is higher than any standard significance level, so we support the null hypothesis. Note that RESET is designed for small-sized samples -- if the sample is big enough, it tends to reject the null hypothesis.

**Exercise 1d** Prepare the first type of diagnostic plots, i.e., plot the residual values vs the fitted values, e.g., using `px.scatter()`. If the assumptions are satisfied, you should see a cloud of points and no trend and the data would be evenly distributed along the horizontal line (add such a line to the graph).

In [ ]:
# put your code here


A useful tweak to this plot is to add a trend line visualizing the relationship between the fitted values and the residuals. This can be easily done with [seaborn.regplot](https://seaborn.pydata.org/generated/seaborn.regplot.html). If the assumptions about linearity are satisfied, you should see a line as close to a horizontal one as possible (minor deviations are acceptable).

In [ ]:
from statsmodels.graphics.gofplots import ProbPlot
import seaborn as sns
import matplotlib.pyplot as plt

# model values
model_fitted_y = results.fittedvalues
# model residuals
model_residuals = results.resid

# here we use matplotlib
# with sns.residplot
# we draw the scatterplot of residuals against the fitted values (scatter=True)
# and we add a regression line
plot_lm_1 = plt.figure()
plot_lm_1.axes[0] = sns.regplot(x=model_fitted_y, y=model_residuals,
                                scatter=True,
                                ci=False,
                                lowess=True,
                                line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_1.axes[0].set_title('Residuals vs Fitted')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals');

**Assumptions about the random disturbance (error term)**

We check the assumptions about the behavior of the distribution of the random disturbance (error term) based on the residuals. Note that we know the relationship between the residuals and the random component thanks to the idempotent matrix M -- the theory presented in the lecture.

***Expected value of the random disturbance***

If the model contains the constant term, there is no need to check the assumption about the expected value of the random disturbance being zero. Because of the properties of the regression hyperplane, the sum of the residuals is zero in the models with constant terms. The mean of the residuals will also be zero, so it is reasonable to use the mean as an estimate of the expected value in this case.

**Exercise 1e** Compare the mean of the residuals in the model with and without the constant. Due to the finite precision of the calculations, you will get a non-zero value in both cases, but this value will be much lower (in terms of absolute values) in the model with a constant.

In [ ]:
# put your code here


***Normality of the distribution of the random disturbance***

We can verify this (additional!) assumption with the graphical analysis of the distributions of the residuals. Graphical analysis of residuals can be done in several ways:

* Histogram/kernel density plot (density plot)
* Quantile-quantile chart
* Boxplot (boxplot)


As here we are working with a relatively large sample -- it is not surprising that we can expect that, under CLT, the distributions of statistics will converge to standard ones.

**Exercise 1f** Inspect the distributions of the residuals in your favorite way. For example, draw the histogram or a qq-plot (see [statsmodels.graphics.gofplots.qqplot](https://www.statsmodels.org/stable/generated/statsmodels.graphics.gofplots.qqplot.html) and explanation [here](https://www.ucd.ie/ecomodel/Resources/QQplots_WebVersion.html) and [here](https://stats.stackexchange.com/questions/101274/how-to-interpret-a-qq-plot)) of the residuals. There should be no significant differences between the empirical and the trend line in the qq-plot, and the histogram should resemble the bell curve. Compare your insights with the conclusion from the Shapiro-Wilk test (e.g., from `scipy.stats`).

In [ ]:
# put your code here (histogram)


In [ ]:
# put your code here (qq-plot)


In [ ]:
# put your code here


***Homoscedasticity and the lack of autocorrelation***

Here, we will use two types of plots: (1) square root of standardized residuals vs the fitted values (also known as scale vs location) and (2) line plot with subsequent residual values.

Let us start with the first type of plots for checking homoscedasticity and lack of autocorrelation: **scale vs location** plot. If the assumptions about the sphericity of the random disturbance are satisfied, we should see a cloud of data in the first graph. If you see triangular shapes of the data clouds, it indicates heteroscedasticity.

**Exercise 1g** Prepare the **scale vs location diagnostic plot**. Plot the squared standardized residuals against the fitted values. Compare with the insights from the first diagnostic plot.

In [ ]:
# put your code here


However, the plot above may be tricky to interpret. That is why you may see a 'tweaked' version of this plot, as shown in the code below. We add a trend line (similarly to the first diagnostic plot) for visibility.

In [ ]:
from statsmodels.graphics.gofplots import ProbPlot
import seaborn as sns
import matplotlib.pyplot as plt

# model values
model_fitted_y = results.fittedvalues
# model residuals
model_residuals = results.resid
# normalized residuals
model_norm_residuals = results.get_influence().resid_studentized_internal
# absolute squared normalized residuals
model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))

plot_lm_2 = plt.figure()
sns.regplot(x=model_fitted_y, y=model_norm_residuals_abs_sqrt,
            scatter=True,
            ci=False,
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8});
plot_lm_2.axes[0].set_title('Scale-Location')
plot_lm_2.axes[0].set_xlabel('Fitted values')
plot_lm_2.axes[0].set_ylabel('$\sqrt{|Standardized Residuals|}$');

 The more horizontal the red line is, the more likely the data is homoscedastic.
If the trend line is horizontal or deviates only slightly, "it's fine." In our case, we see a slight positive deviation between the low and high fit values. However, these are negligible fluctuations. In Exercise 2, you will be able to compare how the trendline behaves when problems occur.

As for the second type of plot for checking homoscedasticity and lack of autocorrelation -- **the line plot with subsequent residual values** -- it allows you to inspect if there are (particularly) problems with autocorrelation. However, you can also spot heteroscedasticity there. If the assumptions about the sphericity of the random component are satisfied, you should see something that resembles a plot of a sound wave with a constant amplitude. The fluctuations should be around zero. Single stronger fluctuations are not a problem.

The problems occur if, for example:
- the amplitude of fluctuations increases (you see a funnel) -- the variance is not constant (heteroscedasticity),
- you notice a trend -- there is autocorrelation,
- periods of high amplitude alternate with periods of low -- unstable variance, "variance clustering" (heteroscedasticity),
- the values do not oscillate around one value -- for example, there is an alternating trend, the graph resembles a sine wave (autocorrelation).

The above list, of course, does not exhaust all possible problems.

**Exercise 1h** Prepare the another diagnostic plot: **the line plot with subsequent residual values**. For convenience, add a horizontal line through zero.

In [ ]:
# put your code here


**Lack of multicolinearity and influential observations**

The last thing -- we need to check if we have the problem of multicollinearity in our model and if there are influential observations. Typically, we say there is a problem if the VIF is about 10 (or the mean VIF is about 10).

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

#calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

print(vif_data)

**Exercise 1i** Interpret the value of VIF in this model.

**Exercise 1j** Find out if there are influential observations (deleating them would noticeably change the model) in the model with `sm.graphics.influence_plot()`. You should be worried if you see the observations in the top right or bottom right part of the plot (this means those observations have, at the same time, high [leverages](https://en.wikipedia.org/wiki/Leverage_(statistics)) and standardized residuals).

In [ ]:
sm.graphics.influence_plot(results, criterion="cooks").show()

## Determining which assumptions are not satisfied

**Exercise 2.** Find out what problems (if any) exist in those models. $n$ is the number of observations, and $k$ is the number of the parameters in the model (including the constant term).

a) n = 204, k = 3
<center><img src='https://mimuw.edu.pl/~dot/resources/wum/diag1.png'>
    </center>

b) n = 1000, k = 4
<center><img src='https://mimuw.edu.pl/~dot/resources/wum/diag2.png'>
    </center>

c) n = 1000, k = 4
<center><img src='https://mimuw.edu.pl/~dot/resources/wum/diag3.png'>
    </center>

d) n = 500, k = 4
<center><img src='https://mimuw.edu.pl/~dot/resources/wum/diag4.png'>
    </center>

e) n = 340, k = 2
<center><img src='https://mimuw.edu.pl/~dot/resources/wum/diag5.png'>
    </center>

**Exercise 3 (homework)** In this exercise, we will predict the income of a municipality in 2020 based on its population and voivodeship.

Create a dataframe with the territorial code, income, population, and voivodeships of municipalities in 2020 using `pd.merge()` to perform a join with the `Code` variable as the key. Remove rows with missing values.
Use the `pd.get_dummies()` function to encode the voivodeship for each municipality with dummy variables.

Estimate the model and inspect its summary.
Are the variables jointly significant according to the F-test?
Are all individual variables significant according to the t-test? What are the interpretations of the parameters?
Can you use a model with intercept in this exercise? Why / why not? If yes, what is its interpretation?

Conduct the diagnostics of the model. Decide which assumptions are satisfied to an appropriate degree.
If you detect an outlying observation, remove it from the data set, rerun the calculations and diagnostics and check if it improves the model fit.

If you detect heteroskedasticity (non-constant variance of residuals), transforming the data may help.
You may transform both the dependent and independent variables.
Transforming the latter changes the functional relationship between the variables (i.e., whether they are linearly related), while transforming the former changes both the relationship and the residual variance structure.

Estimate the average error in PLN that you would make if you used your model to predict a municipality's income from its population.

In [ ]:
# put your code here


## Multicollinearity and omitted variables

**Exercise 4.**  In this exercise, we will show what may happen to our statistical reasoning and the estimates of the parameters if we have multicollinearity in our model or we did not include a significant variable. We will again work with artificial data.

**Exercise 4a** Generate the data ($n=1000$) in a way similar to the one shown in Exercise 1. Our dataset contains the following variables:

- A, which is not correlated with the others, $\mu=0, \sigma^2=1$
- B and C, which are highly correlated (covariance = 24.9875), both with $\mu=20, \sigma^2=25$
- D that will be correlated with variable C (covariance = 8.0), and with B (covariance = 7.99), $\mu=4, \sigma^2=4$

Use seed 23033023.

In [ ]:
# put your code here


**Exercise 4b** Create a variable $y$, which will be the dependent variable in the regression. $ y = 0.3*A + 0.5*B + 0.6*C + \varepsilon$. This way, we will obtain a "real" model. Y will depend on A, B, and C (but not on D). We will also add a small random disturbance (from $N(0,1)$).

In [ ]:
# put your code here


**Exercise 4c** Inspect the descriptive statistics of the data set.
Visualize the relationships between dependent and independent variables. If you had no knowledge about the true model -- which variables would you include?

In [ ]:
# put your code here


**Exercise 4d** Estimate the model and inspect its summary. Are the variables jointly significant according to the F-test? Are all individual variables significant according to the t-test? Compare the estimates against the true values of the parameters. Inspect VIFs.

In [ ]:
# put your code here


**Exercise 4e** Let us include variable D in the model, which we know should be statistically insignificant (it did not participate in creating variable $y$) but is strongly correlated with variable C (statistically significant). Inspect the summary and VIFs.

In [ ]:
# put your code here


**Exercise 4f** Let us construct a model that only includes A, D, and a constant term. D is absent in the true model, and we omitted two significant variables (B and C). Inspect the summary of the model and VIFs.

In [ ]:
# put your code here
